In [27]:
# version de testeo final
# Esta versión se deriva a funciones en el código


import funcionesCalculo as ft
import numpy as np
import matplotlib.pyplot as plt

# importacion de datos del terreno del archivo datos_terreno.xlsx
espesor,cotas,az,nivel_freatico,pe_seco,pe_saturado,E,poisson,cohesion,fi,cc,e0,tipo_datos,tipo_calculo=ft.datos_terreno()


# importación de los datos del terraplen y del mallado obtenido de la excel datos_terraplen.xlsx
a,b,h,q,ax,incrx,incrz=ft.datos_terraplen()

# funciones auxiliares para el cálculo de la presion total

def insertar_valor(lista, valor):
    # funcion para insertar la cota en la lista de cotas y ordenarla
    lista_mod=[] # se evita alterar la lista original
    lista_mod=lista.copy()
    lista_mod.append(valor)  # Agrega el valor a la lista
    lista_mod.sort()         # Ordena la lista en orden ascendente
    return lista_mod


def obtener_maximo_menor(lista, valor):
    # calculamos el valor de la cota inferior del estrato superior 
    # más cercano a la cota que se introduce
    maximo_menor = max(filter(lambda x: x <= valor, lista))
    return maximo_menor


def presion_total(cotas,valor_nf,pe_saturado,pe_seco,valor_cota):
    
    # cotas es la lista de cotas
    # pe_saturado, lista de pesos específicos saturados
    # pe_seco.lista de pesos específicos secos
    # nf es la profundidad del nivel freático
    # z es la profundidad a la que se va a calcular la presión total
    
    
    lista_cotas = cotas.copy() # se copia la lista de las cotas para no alterarla

    #lista ordenada contiene la lista de las cotas y la cota hasta la que se quiere calcular las tensiones
    if valor_nf not in lista_cotas: 
        lista_valores = insertar_valor(lista_cotas, valor_nf)
    else:
        lista_valores=lista_cotas
    
    # resultado es el valor de la cota inmediatamente anterior a la que queremos calcular
    resultado = obtener_maximo_menor(lista_valores, valor_cota)
    
    
    # calculo del estrato del fondo
    peso_saturado=pe_saturado[ft.parametro_terreno(cotas,valor_cota)] # peso especifico saturado
    peso_seco=pe_seco[ft.parametro_terreno(cotas, valor_cota)] # peso especifico seco
    
    # cálculo de las presiones totales y efectivas, inicio de valores
    peso=peso_seco if valor_cota<=valor_nf else peso_saturado
    presion_total=(valor_cota-resultado)*peso
    
    # resto de estratos
    for j in range(lista_valores.index(resultado),0,-1):
        espesor=lista_valores[j]-lista_valores[j-1] # espesor del estrato
        peso_saturado=pe_saturado[ft.parametro_terreno(cotas,lista_valores[j])] # peso especifico saturado
        peso_seco=pe_seco[ft.parametro_terreno(cotas,lista_valores[j])] # peso especifico seco
        posicion=lista_valores[j] # indica la cota del final del nivel
    
        # sumatoria de las presiones
        if (posicion<=valor_nf):
            peso=peso_seco
        else:
            peso=peso_saturado
    
        presion_total=presion_total+espesor*peso
    
    return presion_total


def asiento_consolidacion(hi,cc,e0,t_efectiva,tensionz,cotas,x,z):
    asiento=(hi*cc[ft.parametro_terreno(cotas,z)]/(1+e0[ft.parametro_terreno(cotas,z)]))*np.log10((t_efectiva+tensionz)/t_efectiva)
    return asiento


In [28]:
# vector de tipo de calculo
tipo_calculo

[0, 'c', 'c']

In [29]:
tipo_calculo[ft.parametro_terreno(cotas,5)]

'c'

In [30]:
# prueba de la funcion de comprobacion del tipo de calculo
# se pone a prueba de mayusculas o minúsculas

for z in np.arange(incrz,az+incrz,incrz):
    tipoCalculo=tipo_calculo[ft.parametro_terreno(cotas,z)]
    if (tipoCalculo in ['c','C']):
        print(z,' Consolidacion')
    if (tipoCalculo in ['e','E']):
        print(z,' Cálculo elástico')



0.1  Consolidacion
0.2  Consolidacion
0.30000000000000004  Consolidacion
0.4  Consolidacion
0.5  Consolidacion
0.6  Consolidacion
0.7000000000000001  Consolidacion
0.8  Consolidacion
0.9  Consolidacion
1.0  Consolidacion
1.1  Consolidacion
1.2000000000000002  Consolidacion
1.3000000000000003  Consolidacion
1.4000000000000001  Consolidacion
1.5000000000000002  Consolidacion
1.6  Consolidacion
1.7000000000000002  Consolidacion
1.8000000000000003  Consolidacion
1.9000000000000001  Consolidacion
2.0  Consolidacion
2.1  Consolidacion
2.2  Consolidacion
2.3000000000000003  Consolidacion
2.4000000000000004  Consolidacion
2.5000000000000004  Consolidacion
2.6  Consolidacion
2.7  Consolidacion
2.8000000000000003  Consolidacion
2.9000000000000004  Consolidacion
3.0000000000000004  Consolidacion
3.1  Consolidacion
3.2  Consolidacion
3.3000000000000003  Consolidacion
3.4000000000000004  Consolidacion
3.5000000000000004  Consolidacion
3.6  Consolidacion
3.7  Consolidacion
3.8000000000000003  Consol

In [31]:


# vector de z
asiento=0
x=0

for z in np.arange(incrz,az+incrz,incrz):
    # incrementos de tensiones provocados por la carga del terraplén        
    tensionz=ft.tension_terraplen(a,b,q,x+b,z)[0] # incrementos de tensiones
    
    # tensiones efectivas
    #print('cota ',z)
    # calculo de un valor determinado para testeo
    valor_presion=presion_total(cotas,nivel_freatico,pe_saturado,pe_seco,z)
    #print("El valor de la presion total es {0:.2f} kN/m2".format(valor_presion))
    # valores de las presiones de poro y efectivas
    valor_presion_poro=ft.n_freatico(nivel_freatico,z)*9.81
    #print("El valor de la presion de poro es {0:.2f} kN/m2".format(valor_presion_poro))
    valor_presion_efectiva=valor_presion-valor_presion_poro
    #print("El valor de la presion efectiva es {0:.2f} kN/m2".format(valor_presion_efectiva))
    #print('Incremento de tensiones en z ',tensionz)
    #print('Asiento por consolidacion ')
    #print(asiento_consolidacion(incrz,cc,e0,valor_presion_efectiva,tensionz,cotas,0,z))
    asiento+=asiento_consolidacion(incrz,cc,e0,valor_presion_efectiva,tensionz,cotas,0,z)
print('Asiento total ', asiento)
    

Asiento total  1.8035149347631387


In [2]:
tipo_calculo

[0, 'c']

In [3]:
espesor

[0, 15]

In [9]:
np.arange(incrz,az+incrz,incrz/2)

array([ 0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,  1.2,
        1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,  2.3,
        2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,  3.4,
        3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,  4.5,
        4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,  5.5,  5.6,
        5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,  6.6,  6.7,
        6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,  7.7,  7.8,
        7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,  8.8,  8.9,
        9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,  9.9, 10. ,
       10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11. , 11.1,
       11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12. , 12.1, 12.2,
       12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13. , 13.1, 13.2, 13.3,
       13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14. , 14.1, 14.2, 14.3, 14.4,
       14.5, 14.6, 14.7, 14.8, 14.9, 15. , 15.1])

In [6]:
np.arange(incrz,az+incrz,incrz)

array([ 0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,  1.6,  1.8,  2. ,  2.2,
        2.4,  2.6,  2.8,  3. ,  3.2,  3.4,  3.6,  3.8,  4. ,  4.2,  4.4,
        4.6,  4.8,  5. ,  5.2,  5.4,  5.6,  5.8,  6. ,  6.2,  6.4,  6.6,
        6.8,  7. ,  7.2,  7.4,  7.6,  7.8,  8. ,  8.2,  8.4,  8.6,  8.8,
        9. ,  9.2,  9.4,  9.6,  9.8, 10. , 10.2, 10.4, 10.6, 10.8, 11. ,
       11.2, 11.4, 11.6, 11.8, 12. , 12.2, 12.4, 12.6, 12.8, 13. , 13.2,
       13.4, 13.6, 13.8, 14. , 14.2, 14.4, 14.6, 14.8, 15. ])